In [1]:
import neuroglancer
import numpy as np
import h5py

neuroglancer.set_server_bind_address('127.0.0.1', '12345')
viewer = neuroglancer.Viewer()
dimensions = neuroglancer.CoordinateSpace(
    names=['x', 'y', 'z'],
    units='',
    scales=[1, 1, 1])
with viewer.txn() as s:
    s.dimensions = dimensions
viewer

http://127.0.0.1:12345/v/8ec2ab6e5a0a3fa144a7e0f9c6e0f506fc5d3811/

In [2]:
wormhdf_fn = '/home/ashkan/workspace/deployed/worms_nuclei_metric_learning-deployed/' +\
             'experiments/inst_seg/output76-2/pred/masked-bw=2-bestmodel-step=49000-running_loss=2/worm04.hdf'
with h5py.File(wormhdf_fn, 'r') as f:
    raw = f['raw'][()].astype('float32')
    gt_label = f['label'][()].astype('uint32')  # if in uint format, recognizes as labels
raw = raw / raw.max()
print('raw', raw.shape, raw.min(), raw.max())
print('gt_label', gt_label.shape, gt_label.min(), gt_label.max())

raw (140, 140, 1166) 0.0 1.0
gt_label (140, 140, 1166) 0 533


In [3]:
# for raw data
with viewer.txn() as s:
    s.layers.append(
            name='raw',
            layer=neuroglancer.LocalVolume(
                data=raw,
                volume_type='image',
                dimensions=neuroglancer.CoordinateSpace(
                    names=['x', 'y', 'z'],
                    units=['', '', ''],
                    scales=[1, 1, 1],
                    coordinate_arrays=[
                    None, None, None
                ]),
            voxel_offset=(0, 0, 0)
    ))
    
# for gt label
with viewer.txn() as s:
    s.layers.append(
            name='gt_label',
            layer=neuroglancer.LocalVolume(
                data=gt_label,
                volume_type='segmentation',
                dimensions=neuroglancer.CoordinateSpace(
                    names=['x', 'y', 'z'],
                    units=['', '', ''],
                    scales=[1, 1, 1],
                    coordinate_arrays=[
                    None, None, None
                ]),
            voxel_offset=(0, 0, 0)
    ))

In [12]:
# add another raw image next to previous one and inverse colors with shader
with viewer.txn() as s:
    s.layers.append(
            name='raw',
            layer=neuroglancer.LocalVolume(
                data=raw,
                volume_type='image',
                dimensions=neuroglancer.CoordinateSpace(
                    names=['x', 'y', 'z'],
                    units=['', '', ''],
                    scales=[1, 1, 1],
                    coordinate_arrays=[
                    None, None, None
                ]),
                voxel_offset=(140, 0, 0),
            ),
            shader='''
                void main() {
                  float v = toRaw(getDataValue(0));
                  v= 1.0-v;
                  emitRGBA(vec4(v, 0.0, 0.0, v));
                }
            '''
    )
# check here for other shader ideas: https://github.com/google/neuroglancer/blob/master/src/neuroglancer/sliceview/image_layer_rendering.md

In [22]:
import copy
state = viewer.state
saved_state = copy.deepcopy(state)
print(last_state)
# do some modifications
with viewer.txn() as s:
    s.layers['gt_label'].segments.update([44, 55, 155])
    s.voxel_coordinates = [70, 70, 600]

ViewerState({"dimensions": {"x": [1, ""], "y": [1, ""], "z": [1, ""]}, "position": [70, 70, 600], "crossSectionScale": 0.5040902295748241, "projectionOrientation": [-0.11960987001657486, -0.1577543020248413, 0.11780107021331787, 0.9731032848358154], "projectionScale": 816.1669961410847, "layers": [{"type": "image", "source": "python://volume/8ec2ab6e5a0a3fa144a7e0f9c6e0f506fc5d3811.d589cf186d9c99c78b2d0c9eab89b982592886c9", "name": "raw"}, {"type": "segmentation", "source": "python://volume/8ec2ab6e5a0a3fa144a7e0f9c6e0f506fc5d3811.5a1fce0295d32d5915a443f27dd93e396699b47a", "tab": "segments", "segments": ["155", "170", "44", "444", "55", "56"], "segmentQuery": "56, 170, 444", "name": "gt_label"}, {"type": "image", "source": "python://volume/8ec2ab6e5a0a3fa144a7e0f9c6e0f506fc5d3811.c3d90482d744801013f6ca4bd413a2e2a6cc6f89", "shader": "\n                void main() {\n                  float v = toRaw(getDataValue(0));\n                  v= 1.0-v;\n                  emitRGBA(vec4(v, 0.0, 

In [24]:
# revert back to saved state
viewer.set_state(saved_state)

'53984971ae2a15b353786f220eeaefd7f2960ac3'

### bind action keys

In [25]:
num_actions = 0
def my_action(s):
    global num_actions
    num_actions += 1
    with viewer.config_state.txn() as st:
      st.status_messages['hello'] = ('Got action %d: mouse position = %r' %
                                     (num_actions, s.mouse_voxel_coordinates))
    print('Got my-action')
    print('  Mouse position: %s' % (s.mouse_voxel_coordinates,))
    print('  Layer selected values: %s' % (s.selected_values,))
viewer.actions.add('my-action', my_action)
with viewer.config_state.txn() as s:
    s.input_event_bindings.viewer['keyt'] = 'my-action'
    s.status_messages['hello'] = 'Welcome to this example'

Got my-action
  Mouse position: [ 86.350204  83.26     584.37317 ]
  Layer selected values: Map({"raw": {"value": 0.3694581389427185}, "gt_label": {"value": "339"}, "raw1": {}})


### screen shot

In [26]:
from ipywidgets import Image
screenshot = viewer.screenshot(size=[1000, 1000])
screenshot_image = Image(value=screenshot.screenshot.image)
screenshot_image

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x05\xa0\x00\x00\x035\x08\x06\x00\x00\x00\x86\xdbV\x0…

### change layout

In [27]:
with viewer.txn() as s:
    s.layout = neuroglancer.row_layout(
        [neuroglancer.LayerGroupViewer(layers=['raw', 'gt_label']),
         neuroglancer.LayerGroupViewer(layers=['gt_label'])])

In [28]:
neuroglancer.stop()